In [11]:
import pandas as pd
import numpy as np

In [115]:
#Read tutee file
tutee_df = pd.read_csv('sample_tutee.csv')

#Format tutee file to be more readable
#Change column names
new_columns = {
    "Kindlespark Point-of-Contact" : "poc",
    "Tutee Index" : "tutee_index",
    "Gender" : "gender",
    "Level in 2023" : "level",
    "Subject #1" : "subject1",
    "Subject #2" : "subject2",
    "Subject #3" : "subject3",
    "Financial Aid?" : "fa",
    "Available?" : "available",
    "Comments" : "comments"
}
tutee_df = tutee_df.rename(columns=new_columns)
tutee_df.head()

#simplify levels to p (primary) and s (secondary)
#fa and available from yes/no to y/n
def level(x):
    return np.char.lower(x[0])

tutee_df['level'] = tutee_df['level'].apply(lambda x: level(x))
tutee_df['fa'] = tutee_df['fa'].apply(lambda x: level(x))
tutee_df['available'] = tutee_df['available'].apply(lambda x: level(x))

#fill na values with nothing
tutee_df[['subject1','subject2','subject3']] = tutee_df[['subject1','subject2','subject3']].fillna('')
tutee_df.head()

,poc,tutee_index,gender,level,subject1,subject2,subject3,fa,available,comments
0,Benjamin,233,Male,s,science,english,,y,y,NaN
1,Benjamin,254,Female,p,english,science,,y,y,Not available for tuition on Wednesdays
2,Dawne,260,Female,p,english,math,science,y,y,"Tutees 260 & 261 are sisters, parent requested..."
3,Dawne,261,Female,p,english,math,science,y,y,"Tutees 260 & 261 are sisters, parent requested..."
4,Benjamin,280,Female,p,english,math,malay,y,y,NaN


In [116]:
#Read tutee file
tutor_df = pd.read_csv('sample_tutor.csv')

#fill na values with nothing
tutee_df[['subject1','subject2']] = tutee_df[['subject1','subject2']].fillna('')
tutor_df.head()

,tutor_index,gender,level,subject1,subject2,probono,available
0,1,m,p,science,english,y,y
1,2,m,p,english,science,y,y
2,3,f,p,english,math,y,y
3,4,f,p,english,math,y,y
4,5,m,p,english,math,y,y


In [133]:
#Do an inner merge between tutors and tutee
#i.e. return rows based on 3 conditions
#1. The level the tutor teaches is the same as the tutee
#2. Pro bono tutees are matched with financial assistance tutees and the converse is true
#3. Both tutor and tutee are available
match_df = pd.merge(tutor_df,tutee_df, how='inner',left_on=['level','probono','available'],right_on=['level','fa','available'])
match_df.head()

,tutor_index,gender_x,level,subject1_x,subject2_x,probono,available,poc,tutee_index,gender_y,subject1_y,subject2_y,subject3,fa,comments
0,1,m,p,science,english,y,y,Benjamin,254,Female,english,science,,y,Not available for tuition on Wednesdays
1,1,m,p,science,english,y,y,Dawne,260,Female,english,math,science,y,"Tutees 260 & 261 are sisters, parent requested..."
2,1,m,p,science,english,y,y,Dawne,261,Female,english,math,science,y,"Tutees 260 & 261 are sisters, parent requested..."
3,1,m,p,science,english,y,y,Benjamin,280,Female,english,math,malay,y,NaN
4,1,m,p,science,english,y,y,Anthony,284,Male,english,chinese,,y,Tutees 284 and 285 are brothers


In [162]:
#This function matches the subjects the tutor teaches and the tutee receives
#Returns the total matched subjects as a new column "matched_subjects"
def match_subjects(df):
    subjects_list = []
    #check for na values
    tutor_subjects = [df['subject1_x'],df['subject2_x']]
    tutee_subjects = [df['subject1_y'],df['subject2_y'],df['subject3']]
    #Gives set of matched subjects
    matched = set(tutor_subjects).intersection(tutee_subjects)
    return ",".join(matched) if matched != set('') else np.nan
    
match_df['matched_subjects'] = match_df.apply(lambda df : match_subjects(df), axis=1)

#Take only relavant information to output
truncated_df = match_df[['tutor_index','tutee_index','matched_subjects','poc','comments']].dropna(axis=0,subset=['matched_subjects'])

#dictionary of more readable names
readable_names = {
    'tutor_index' : "Tutor Index",
    'tutee_index' : "Tutee Index",
    'matched_subjects' : 'Matching Subjects',
    'poc' : 'Kindlespark Point-of-Contact',
    "comments" : "Comments"
}
truncated_df = truncated_df.rename(columns = readable_names)

#Group by tutors for clarity
truncated_df = truncated_df.set_index('Tutor Index', append=True).swaplevel(0,1)

#save df
truncated_df.to_excel('sample_output.xlsx')

,Tutor Index,Tutee Index,Matching Subjects,Kindlespark Point-of-Contact,Comments
0,1,254,"science,english",Benjamin,Not available for tuition on Wednesdays
1,1,260,"science,english",Dawne,"Tutees 260 & 261 are sisters, parent requested..."
2,1,261,"science,english",Dawne,"Tutees 260 & 261 are sisters, parent requested..."
3,1,280,english,Benjamin,NaN
4,1,284,english,Anthony,Tutees 284 and 285 are brothers
...,...,...,...,...,...
159,11,286,"e-math,english",Dawne,"Tutees 286, 287 and 288 are siblings"
160,11,287,"e-math,english",Dawne,"Tutees 286, 287 and 288 are siblings"
161,11,289,e-math,Dawne,NaN
162,11,293,e-math,Anthony,Tutees 293 & 294 are siblings
